In [1]:
import re
import os
import pickle
import numpy as np
import logging
import torch
import math
import time
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader
from collections import defaultdict

from transformers import BertTokenizer,BertModel,BertTokenizerFast
from utils.MIND import MIND

logger = logging.getLogger(__name__)

In [2]:
# config.reducer = 'bm25'
# config.reducer = 'bow'
# config.reducer = 'matching'

# config.signal_length = 10
# config.scale = 'large'
config.impr_size = 100
config.mode = 'test'
path = config.path + 'MIND/MINDdemo_test/'
a = MIND(config, path + 'news.tsv', path + 'behaviors.tsv')

[2021-08-30 13:19:41,459] INFO (utils.MIND) encoding user behaviors of ../../../Data/MIND/MINDdemo_test/behaviors.tsv...
[2021-08-30 13:19:41,535] INFO (utils.MIND) loading cached news tokenization from data/cache/bert/MINDdemo_test/news.pkl
[2021-08-30 13:19:42,691] INFO (utils.MIND) reducing news of ../../../Data/MIND/MINDdemo_test/news.tsv...
[2021-08-30 13:19:42,693] INFO (utils.utils) unmasking at least k...
[2021-08-30 13:19:43,306] INFO (utils.utils) deduplicating...


In [ ]:
a[0]['cdd_subword_index'].shape

In [34]:
list(p1), list(p2), list(p3), list(p4),  len(p1), len(p2), len(p3), len(p4)

([0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14],
 [15, 16, 17, 18, 19, 20],
 5,
 5,
 5,
 6)

In [3]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=Partition_Sampler(a,2,0))
records1 = list(loader1

In [7]:
records1[0]['his_attn_mask']

tensor([[[1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [ ]:
dic = pickle.load(open('/data/workspace/Peitian/Code/Document-Reduction/Code/data/cache/bert/MINDdemo_dev/10/behaviors.pkl', 'rb'))
dic['imprs']

In [2]:
t = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [4]:
t.tokenize("I don't give a fuck")

['i', 'don', "'", 't', 'give', 'a', 'fuck']